In [18]:
import polars as pl
from scipy import stats

In [19]:
thepath = r'C:\Users\MARKETING SUPPORT\Documents\asteros\astrs_work\on_prog_q4_explore.parquet'
masters = pl.read_parquet(thepath)

In [20]:
revenue = masters.filter([pl.col('Status Pesanan')!="Batal",pl.col('Waktu Pengiriman Diatur').is_not_null()]).sort(by='Waktu Pengiriman Diatur')['No. Pesanan','Status Pesanan','Alasan Pembatalan','Status Pembatalan/ Pengembalian','No. Resi',
       'Opsi Pengiriman','Antar ke counter/ pick-up','Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan)','Waktu Pengiriman Diatur',
       'Waktu Pesanan Dibuat','Waktu Pembayaran Dilakukan','Metode Pembayaran','SKU Induk','Nama Produk','Nomor Referensi SKU','Nama Variasi',
       'Harga Awal','Harga Setelah Diskon','Jumlah','Returned quantity','Total Harga Produk','Total Diskon','Diskon Dari Penjual',
       'Diskon Dari Shopee','Berat Produk','Jumlah Produk di Pesan','Total Berat','Voucher Ditanggung Penjual','ditanggung penjual no dup',
       'Cashback Koin','Voucher Ditanggung Shopee','Paket Diskon','Paket Diskon (Diskon dari Shopee)','Paket Diskon (Diskon dari Penjual)',
       'Potongan Koin Shopee','Diskon Kartu Kredit','Ongkos Kirim Dibayar oleh Pembeli','Estimasi Potongan Biaya Pengiriman','Ongkos Kirim Pengembalian Barang',
       'Total Pembayaran','total no dup','Perkiraan Ongkos Kirim','Catatan dari Pembeli','Catatan','Username (Pembeli)','Nama Penerima',
       'No. Telepon','Alamat Pengiriman','Kota/Kabupaten','Provinsi','Waktu Pesanan Selesai','source',
 ]
groupdate = revenue.with_columns(pl.col('Waktu Pesanan Dibuat').dt.date().alias('Group_date').cast(pl.Utf8)).filter(pl.col('Group_date')<'2024-11-01')

# executive

In [21]:
groupdate.group_by('Group_date').agg(pl.col('total no dup').sum()).sort('Group_date').plot.line(x='Group_date',y='total no dup')

alt.Chart(...)

### insight 

if we penalize x factor like event 10. 10 shopee the first 2 week i goes rally start from 4th day of month before they falling off

In [22]:
import altair as alt

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [23]:
corr = groupdate.group_by('Group_date').agg([pl.col('total no dup').sum().alias('total rev'),pl.col('ditanggung penjual no dup').sum().alias('total discount')]).sort(by='Group_date')

corr.plot.scatter(y='total rev',x='total discount')

alt.Chart(...)

In [24]:
var1 = groupdate['total no dup'].to_numpy()
var2 = groupdate['ditanggung penjual no dup'].to_numpy()

corr , pval = stats.spearmanr(var1,var2)

print(f'correlation value"s are {corr} and pvalue is {pval:.5f}')

correlation value"s are 0.6117287488996367 and pvalue is 0.00000


In [26]:
corr*corr*100

np.float64(37.421206223031476)

### insight

whenever our store gave discount on the product the revenue tends to follow that eg. when our revenue is 1000000 around 37% (its squared of correlation value) of it can be achive because of discount strategies move by our company

In [27]:
groupdate

No. Pesanan,Status Pesanan,Alasan Pembatalan,Status Pembatalan/ Pengembalian,No. Resi,Opsi Pengiriman,Antar ke counter/ pick-up,Pesanan Harus Dikirimkan Sebelum (Menghindari keterlambatan),Waktu Pengiriman Diatur,Waktu Pesanan Dibuat,Waktu Pembayaran Dilakukan,Metode Pembayaran,SKU Induk,Nama Produk,Nomor Referensi SKU,Nama Variasi,Harga Awal,Harga Setelah Diskon,Jumlah,Returned quantity,Total Harga Produk,Total Diskon,Diskon Dari Penjual,Diskon Dari Shopee,Berat Produk,Jumlah Produk di Pesan,Total Berat,Voucher Ditanggung Penjual,ditanggung penjual no dup,Cashback Koin,Voucher Ditanggung Shopee,Paket Diskon,Paket Diskon (Diskon dari Shopee),Paket Diskon (Diskon dari Penjual),Potongan Koin Shopee,Diskon Kartu Kredit,Ongkos Kirim Dibayar oleh Pembeli,Estimasi Potongan Biaya Pengiriman,Ongkos Kirim Pengembalian Barang,Total Pembayaran,total no dup,Perkiraan Ongkos Kirim,Catatan dari Pembeli,Catatan,Username (Pembeli),Nama Penerima,No. Telepon,Alamat Pengiriman,Kota/Kabupaten,Provinsi,Waktu Pesanan Selesai,source,Group_date
str,str,str,null,str,str,str,datetime[ms],datetime[ms],datetime[ms],datetime[ms],str,null,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,null,null,str,str,str,str,str,str,datetime[ms],str,str
"""241001CUKMTPHP""","""Selesai""",null,null,"""GK-11-784428416""","""Instant-GoSend Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 10:58:00,2024-10-01 07:32:00,2024-10-01 07:32:00,"""ShopeePay""",null,"""UGREEN Adaptor Kepala Charger …","""10220""","""ADP 20W MINI WHITE""",119000,119000,1,0,119000,0,0,0,"""250 gr""",1,"""250 gr""",0,0,0,2380,"""N""",0,0,0,0,0,18000,0,118620,118620,18000,null,null,"""nfiata""","""N***a""","""******53""","""Jl. Batang Hari No.6, RT.5/RW.…","""KOTA JAKARTA PUSAT""","""DKI JAKARTA""",2024-10-09 04:29:00,"""10_2024-oct (1).xlsx""","""2024-10-01"""
"""241001CTY0TPCF""","""Selesai""",null,null,"""2251004833009152369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:28:00,2024-10-01 07:21:00,2024-10-01 07:21:00,"""ShopeePay""",null,"""UGREEN Charger iPhone MFi USB …","""50460-60161""","""1SET 10W WHITE NYLON""",212500,212500,1,0,212500,0,0,0,"""200 gr""",1,"""200 gr""",0,0,0,21250,"""N""",0,0,0,0,0,20000,0,211250,211250,20000,null,null,"""zahrah.mutiara""","""Z******a""","""******33""","""Jalan Kp Rawa Sawah No.7, RT.2…","""KOTA JAKARTA PUSAT""","""DKI JAKARTA""",2024-10-01 17:14:00,"""10_2024-oct (1).xlsx""","""2024-10-01"""
"""241001D0D47A0N""","""Selesai""",null,null,"""2251004910424064369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:39:00,2024-10-01 08:58:00,2024-10-01 08:59:00,"""Online Payment""",null,"""UGREEN USB Hub 10 in 1 Type C …","""15601""","""15601 10 IN 1""",490000,490000,3,0,1470000,0,0,0,"""900 gr""",3,"""900 gr""",0,0,0,0,"""N""",0,0,0,0,0,28000,0,1486400,1486400,28000,null,null,"""hestisusanti20""","""H******i""","""******14""","""Ruko Bona Gabe, Jalan Jatinega…","""KOTA JAKARTA TIMUR""","""DKI JAKARTA""",2024-10-07 15:07:00,"""10_2024-oct (1).xlsx""","""2024-10-01"""
"""241001D04YUDH1""","""Selesai""",null,null,"""2251005068414464369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:40:00,2024-10-01 08:54:00,2024-10-01 08:54:00,"""ShopeePay""",null,"""UGREEN Kabel Data MFI iPhone 5…","""60158""","""60158 2M""",175000,175000,1,0,175000,0,0,0,"""200 gr""",1,"""200 gr""",2500,2500,0,13800,"""N""",0,0,0,0,0,18000,0,162400,162400,18000,null,null,"""indahhhs.61""","""w******l""","""******63""","""Warung Pak Haji Alek/ Bang Zul…","""KOTA JAKARTA BARAT""","""DKI JAKARTA""",2024-10-01 23:49:00,"""10_2024-oct (1).xlsx""","""2024-10-01"""
"""241001CUY3WU7E""","""Selesai""",null,null,"""2251005018542080369""","""Instant-SPX Instant""","""Pickup""",2024-10-01 23:59:00,2024-10-01 11:46:00,2024-10-01 07:39:00,2024-10-01 07:39:00,"""ShopeePay""",null,"""UGREEN Microphone Desktop USB …","""10934""","""UGREEN 10934""",679000,265000,1,0